# SQL Lite Reference
## Analysis of Seinfeld episodes and ratings
### with DataFrame equivallent implementations

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import pprint
pp = pprint.PrettyPrinter()

In [2]:
ratings_file = 'data/seinfeld/ratings.csv'
ratings_df = pd.read_csv(ratings_file)

print ('\n-- Ratings: ', ratings_df.shape)
display(ratings_df.head(3))

episodes_file = 'data/seinfeld/episode_info.csv'
episodes_df = pd.read_csv(episodes_file)
episodes_df = episodes_df.drop('Unnamed: 0', axis=1)
print ('\n-- Episodes ', episodes_df.shape)
display(episodes_df.head(3))



-- Ratings:  (172, 5)


,Season,Episode,Title,Rating,Votes
0,1,2,The Stakeout,7.8,649
1,1,3,The Robbery,7.7,565
2,1,4,Male Unbonding,7.6,561



-- Episodes  (174, 7)


,Season,EpisodeNo,Title,AirDate,Writers,Director,SEID
0,1.0,1.0,"Good News, Bad News","July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E01
1,1.0,1.0,The Stakeout,"May 31, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E01
2,1.0,2.0,The Robbery,"June 7, 1990",Matt Goldman,Tom Cherones,S01E02


In [3]:
# cleanup
episodes_df2 = episodes_df.copy()
episodes_df2['Director'] = episodes_df['Director'].str.replace('&nbsp;', ' ')
episodes_df2['Director'] = episodes_df2['Director'].replace(to_replace='[ ]+', value=' ', regex=True)
episodes_df2['Director'].unique()

array(['Art Wolff', 'Tom Cherones', 'David Steinberg', 'Joshua White',
       'Jason Alexander', 'Andy Ackerman', 'David Owen Trainor'],
      dtype=object)

In [4]:
# create table

sql = ''' create table ratings
(SEASON INTEGER,
 EPISODE INTEGER,
 TITLE TEXT,
 RATING REAL,
 VOTES INTEGER);
'''

seindb = 'seinfeld.db'

conn = sqlite3.connect(seindb)
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS ratings")

cursor.execute(sql)
print (f'table ratings in database {seindb} created')

sql = ''' create table episodes
(SEASON INTEGER,
 EPISODE INTEGER,
 TITLE TEXT,
 AIRDATE DATE,
 WRITERS TEXT,
 DIRECTOR TEXT,
 SEID TEXT);
'''
cursor.execute("DROP TABLE IF EXISTS episodes")
cursor.execute(sql)
print (f'table episodes in database {seindb} created')

conn.commit()

table ratings in database seinfeld.db created
table episodes in database seinfeld.db created


In [5]:
# insert entries
ratings_data = ratings_df.values
print (f'adding {len(ratings_data)} records to ratings table')
sql = 'INSERT INTO ratings VALUES (?, ?, ?, ?, ?)'
cursor.executemany(sql, ratings_data)

episodes_data = episodes_df.values
#print(episodes_data[:10])
print (f'adding {len(episodes_data)} records to episodes table')
sql = 'INSERT INTO episodes VALUES (?, ?, ?, ?, ?, ?, ?)'
cursor.executemany(sql, episodes_data)
conn.commit() # commit is on connection


adding 172 records to ratings table
adding 174 records to episodes table


In [6]:
# reading from db - limit 10 rows
cursor.execute('select * from ratings limit 10')
for n, row in enumerate(cursor):
        print(n, row)


0 (1, 2, 'The Stakeout', 7.8, 649)
1 (1, 3, 'The Robbery', 7.7, 565)
2 (1, 4, 'Male Unbonding', 7.6, 561)
3 (1, 5, 'The Stock Tip', 7.8, 541)
4 (2, 1, 'The Ex-Girlfriend', 7.7, 529)
5 (2, 1, 'The Statue', 8.1, 509)
6 (2, 11, 'The Heart Attack', 8.0, 512)
7 (2, 12, 'The Revenge', 8.4, 497)
8 (2, 13, 'The Deal', 8.3, 515)
9 (2, 2, 'The Pony Remark', 8.0, 573)


In [7]:
# read into DF - using where 
# note - you pass connection not cursor here.
df = pd.read_sql_query("SELECT * from ratings where rating > 8", conn)
df.head(3)
#df.shape

,SEASON,EPISODE,TITLE,RATING,VOTES
0,2,1,The Statue,8.1,509
1,2,12,The Revenge,8.4,497
2,2,13,The Deal,8.3,515


In [8]:
# read from table into dataframe
df = pd.read_sql_query("SELECT * from episodes", conn)
df.head(3)
#df.shape

,SEASON,EPISODE,TITLE,AIRDATE,WRITERS,DIRECTOR,SEID
0,1,1,"Good News, Bad News","July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E01
1,1,1,The Stakeout,"May 31, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E01
2,1,2,The Robbery,"June 7, 1990",Matt Goldman,Tom Cherones,S01E02


In [9]:
# join multiple tables.
# note distinct - otherwise dup rows returned
sql = '''
select distinct episodes.seid, ratings.title, episodes.director, ratings.rating 
from ratings, episodes
where ratings.title = episodes.title and
ratings.rating >= 8.7
'''

# alternate way to run query
for match in cursor.execute(sql):
    print (match)
# what the heck is this returning ? 

('S02E11', 'The Chinese Restaurant', 'Tom Cherones', 8.7)
('S03E13', 'The Subway', 'Tom Cherones', 8.7)
('S03E05', 'The Library', 'Joshua White', 8.7)
('S04E11', 'The Contest', 'Tom Cherones', 8.7)
('S04E19', 'The Implant', 'Tom Cherones', 8.7)
('S04E21', 'The Smelly Car', 'Tom Cherones', 8.7)
('S04E03', 'The Pitch', 'Tom Cherones', 8.7)
('S04E07', 'The Bubble Boy', 'Tom Cherones', 8.7)
('S05E20', 'The Fire', 'Tom Cherones', 8.7)
('S07E11', 'The Rye', 'Andy Ackerman', 8.7)
('S07E24', 'The Invitations', 'Andy Ackerman', 8.7)
('S07E04', 'The Wink', 'Andy Ackerman', 8.7)
('S07E05', 'The Hot Tub', 'Andy Ackerman', 8.7)
('S07E06', 'The Soup Nazi', 'Andy Ackerman', 8.7)
('S08E13', 'The Comeback', 'David Owen Trainor', 8.7)
('S08E04', 'The Little Kicks', 'Andy Ackerman', 8.7)


In [10]:
# doing the same with DF
out_df = pd.merge(episodes_df[['SEID', 'Title', 'Director']], ratings_df[['Title', 'Rating']], on = 'Title', how = 'right')
print(out_df.shape)
display(out_df.head(3))
out_df[out_df['SEID'] == 'S02E11']

(172, 4)


,SEID,Title,Director,Rating
0,S01E01,The Stakeout,Tom Cherones,7.8
1,S01E02,The Robbery,Tom Cherones,7.7
2,S01E03,Male Unbonding,Tom Cherones,7.6


,SEID,Title,Director,Rating
13,S02E11,The Chinese Restaurant,Tom Cherones,8.7


In [11]:
# Pulling data from another DF. 
# use merge with subset of columns.
display(ratings_df.head(3))
ratings_df_rows = ratings_df.shape[0]
display(episodes_df.head(3))

ratings_df2 = pd.merge(ratings_df, episodes_df[['Title', 'SEID']], on='Title', how='left')
ratings_df2_rows = ratings_df2.shape[0]
assert ratings_df_rows == ratings_df2_rows

,Season,Episode,Title,Rating,Votes
0,1,2,The Stakeout,7.8,649
1,1,3,The Robbery,7.7,565
2,1,4,Male Unbonding,7.6,561


,Season,EpisodeNo,Title,AirDate,Writers,Director,SEID
0,1.0,1.0,"Good News, Bad News","July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E01
1,1.0,1.0,The Stakeout,"May 31, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E01
2,1.0,2.0,The Robbery,"June 7, 1990",Matt Goldman,Tom Cherones,S01E02


In [12]:
# join tables.
sql = '''
select ratings.title, episodes.director, ratings.rating 
from ratings
left join episodes on ratings.title = episodes.title and
ratings.season = episodes.season 
limit 10
'''
cursor.execute(sql)
matches = cursor.fetchall()
for i, match in enumerate(matches):
    print (i, match)

0 ('The Stakeout', 'Tom Cherones', 7.8)
1 ('The Robbery', 'Tom Cherones', 7.7)
2 ('Male Unbonding', 'Tom Cherones', 7.6)
3 ('The Stock Tip', 'Tom Cherones', 7.8)
4 ('The Ex-Girlfriend', 'Tom Cherones', 7.7)
5 ('The Statue', 'Tom Cherones', 8.1)
6 ('The Heart Attack', 'Tom Cherones', 8.0)
7 ('The Revenge', 'Tom Cherones', 8.4)
8 ('The Deal', 'Tom Cherones', 8.3)
9 ('The Pony Remark', 'Tom Cherones', 8.0)


In [13]:
# aggregation - get avg rating by season.
query = '''
select season, avg(rating)
from ratings
group by season
'''
for match in cursor.execute(query):
    print (match)

(1, 7.7250000000000005)
(2, 8.158333333333333)
(3, 8.304347826086957)
(4, 8.465217391304348)
(5, 8.342857142857143)
(6, 8.282608695652174)
(7, 8.44090909090909)
(8, 8.422727272727274)
(9, 8.322727272727274)


In [14]:
ratings_df.groupby('Season')['Rating'].mean()

Season
1    7.725000
2    8.158333
3    8.304348
4    8.465217
5    8.342857
6    8.282609
7    8.440909
8    8.422727
9    8.322727
Name: Rating, dtype: float64

In [15]:
# get episodes/season
# count(*) -- count number of rows
query = '''
select season,count(*), max(rating)
from ratings
group by season
'''
for match in cursor.execute(query):
    print (match)

(1, 4, 7.8)
(2, 12, 8.7)
(3, 23, 8.7)
(4, 23, 8.7)
(5, 21, 8.7)
(6, 23, 8.6)
(7, 22, 8.7)
(8, 22, 8.7)
(9, 22, 8.6)


In [16]:
# doing same with DF
out_df = ratings_df.groupby(['Season'])[['Rating']].count()
out_df = out_df.reset_index()
#out_df.columns = [' '.join(col).strip() for col in out_df.columns.values]
print(out_df.columns)
out_df

Index(['Season', 'Rating'], dtype='object')


,Season,Rating
0,1,4
1,2,12
2,3,23
3,4,23
4,5,21
5,6,23
6,7,22
7,8,22
8,9,22


In [17]:
ratings_df.columns
df = ratings_df[ratings_df['Season']==2]
df

,Season,Episode,Title,Rating,Votes
4,2,1,The Ex-Girlfriend,7.7,529
5,2,1,The Statue,8.1,509
6,2,11,The Heart Attack,8.0,512
7,2,12,The Revenge,8.4,497
8,2,13,The Deal,8.3,515
9,2,2,The Pony Remark,8.0,573
10,2,3,The Busboy,7.9,500
11,2,4,The Baby Shower,7.8,504
12,2,5,The Jacket,8.5,560
13,2,6,The Chinese Restaurant,8.7,656


In [18]:
# select  unique entries
query = 'select distinct director from episodes'
df = pd.read_sql_query(query, conn)
display(df)

,DIRECTOR
0,Art Wolff
1,Tom Cherones
2,David Steinberg
3,Joshua White
4,David&nbsp; Steinberg
5,Jason Alexander
6,Andy Ackerman
7,David Owen Trainor


In [19]:
# same with DF
# cleanedup data detects one dup entry correctly (David Steinberg)
episodes_df2.Director.unique()

array(['Art Wolff', 'Tom Cherones', 'David Steinberg', 'Joshua White',
       'Jason Alexander', 'Andy Ackerman', 'David Owen Trainor'],
      dtype=object)

In [20]:
# group by , having, order by
# get number of episodes / director sorted
query = '''select director, count(title) as NUM_EPISODES
from episodes
group by director
having NUM_EPISODES > 1
order by NUM_EPISODES desc
'''
df = pd.read_sql_query(query, conn)
display(df.shape, df)

(3, 2)

,DIRECTOR,NUM_EPISODES
0,Andy Ackerman,87
1,Tom Cherones,80
2,David Owen Trainor,2


In [21]:
# same in df/series
s = episodes_df2.groupby(['Director'])['Title'].count()
s_sorted = s[s>1].sort_values(ascending=False)
print(s_sorted)

Director
Andy Ackerman         87
Tom Cherones          80
David Steinberg        2
David Owen Trainor     2
Name: Title, dtype: int64


In [22]:
# what does collate do ? 
query = '''select director, count(title) as NUM_EPISODES
from episodes
group by director collate nocase
order by NUM_EPISODES desc
'''
df = pd.read_sql_query(query, conn)
display(df.shape, df)

(8, 2)

,DIRECTOR,NUM_EPISODES
0,Andy Ackerman,87
1,Tom Cherones,80
2,David Owen Trainor,2
3,Joshua White,1
4,Jason Alexander,1
5,David&nbsp; Steinberg,1
6,David Steinberg,1
7,Art Wolff,1


In [23]:
# search by in
query = '''
select distinct title, rating 
from ratings
where lower(title) in ("the deal", "the apology", "male unbonding")
'''

df = pd.read_sql_query(query, conn)
display(df)

,TITLE,RATING
0,Male Unbonding,7.6
1,The Deal,8.3
2,The Apology,8.2


In [24]:
# substring search / -rating => order by rating desc
query = '''
select distinct lower(title), rating 
from ratings
where lower(title) like '%boy%'
order by -rating
'''

df = pd.read_sql_query(query, conn)
display(df.shape, df.head(10))

(5, 2)

,lower(title),RATING
0,the boyfriend: part 2,8.7
1,the bubble boy,8.7
2,the boyfriend: part 1,8.6
3,the van buren boys,8.2
4,the busboy,7.9


## Using JOIN to join two queries.
### work on individual queries seperately 
### WHERE and JOIN can be used in some cases - looks like where is nothing but implicit JOIN
### Ref: https://dataschool.com/how-to-teach-people-sql/difference-between-where-and-on-in-sql/

In [25]:
# For each director, get their worst and best show.
# output should have:
# Director, Min_Title, Min_Rating, Max_Title, Max_Rating
# where Min_Title is the one of the titles with minimum rating 
# (you can sort it by rating ?)


# tmax and tmin are two tables consturcted on the fly
# and we join on common column
query = '''
select tmax.director,
    tmax.title as max_title,
    tmax.max_rating,
    tmin.title as min_title,
    tmin.min_rating

from

    ( select episodes.director, 
        episodes.title,
        max(ratings.rating) as max_rating
      from episodes, ratings
      where episodes.title == ratings.title
      group by episodes.director ) tmax

join 

    ( select episodes.director, 
        episodes.title,
        min(ratings.rating) as min_rating
    from episodes, ratings
    where episodes.title == ratings.title
    group by episodes.director ) tmin
on tmax.director == tmin.director
'''

query1 = '''
select episodes.director, 
    episodes.title,
    min(ratings.rating) as min_rating
from episodes, ratings
where episodes.title == ratings.title
group by episodes.director
'''

query1 = '''
select episodes.director, 
    episodes.title,
    max(ratings.rating) as max_rating
from episodes, ratings
where episodes.title == ratings.title
group by episodes.director
'''
query2 = '''
    select min(rating), max(rating)
    from ratings
'''

query2 = '''
    select * from ratings where lower(title) = 'the wink' 
'''

df = pd.read_sql_query(query, conn)
display(df.shape, df.head(10))

(7, 5)

,director,max_title,max_rating,min_title,min_rating
0,Andy Ackerman,The Wink,8.7,The Maid,7.9
1,David Owen Trainor,The Comeback,8.7,The Secretary,8.1
2,David Steinberg,The Truth,7.8,The Truth,7.8
3,David&nbsp; Steinberg,The Tape,8.6,The Tape,8.6
4,Jason Alexander,The Good Samaritan,8.1,The Good Samaritan,8.1
5,Joshua White,The Library,8.7,The Library,8.7
6,Tom Cherones,The Chinese Restaurant,8.7,The Dog,7.3


In [26]:
# doing same with DF
# not optimal implementation
# merge episodes and ratings and together
merged_df = pd.merge(episodes_df2[['Title', 'Director']], ratings_df2[['Title', 'Rating']], how='right', on='Title')
#print ('--- merged df ---'); display(merged_df.head(3))

# get min and max indexes into this table for each director. 
# this index is into merged_df
# idxmin() get ID witih min value.
min_idx = merged_df.groupby(['Director'])[['Rating']].idxmin().reset_index()
max_idx = merged_df.groupby(['Director'])[['Rating']].idxmax().reset_index()
min_idx.columns = ['DIRECTOR', 'MIN_IDX']
max_idx.columns = ['DIRECTOR', 'MAX_IDX']
# merge them both 
result_df = pd.merge(max_idx.reset_index(drop=True), min_idx.reset_index(drop=True))
#print ('-- MinMax Index'); display(result_df)

# Now get the title and rating for each of the index from merged_df.
result_df[['MAX_TITLE', 'MAX_RATING']] = result_df['MAX_IDX'].apply (lambda x : merged_df.loc[x][['Title', 'Rating']])
result_df[['MIN_TITLE', 'MIN_RATING']] = result_df['MIN_IDX'].apply (lambda x : merged_df.loc[x][['Title', 'Rating']])
# drop unwanted columns
result_df = result_df.drop(['MIN_IDX', 'MAX_IDX'], axis = 1)
display(result_df)


,DIRECTOR,MAX_TITLE,MAX_RATING,MIN_TITLE,MIN_RATING
0,Andy Ackerman,The Rye,8.7,The Maid,7.9
1,David Owen Trainor,The Comeback,8.7,The Secretary,8.1
2,David Steinberg,The Tape,8.6,The Truth,7.8
3,Jason Alexander,The Good Samaritan,8.1,The Good Samaritan,8.1
4,Joshua White,The Library,8.7,The Library,8.7
5,Tom Cherones,The Chinese Restaurant,8.7,The Dog,7.3


In [27]:
# cleanup
cursor.close()
conn.close()